In [1]:
####!/usr/bin/python
##### coding: utf-8

In [2]:
#Action Plan

#1-Get and prepare data
#2-Normalize Data
#3-Processing Data
#4-Prepare Data to DDBB
#5-Upload data to DDBB

In [3]:
#Import required libraries

import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import normalize
import psycopg2, psycopg2.extras
import psycopg2.extensions
import sys
psycopg2.extensions.register_type(psycopg2.extensions.UNICODE) 
psycopg2.extensions.register_type(psycopg2.extensions.UNICODEARRAY)

#See al columns
#pd.set_option('display.max_columns',50).


### 1-Get Data

In [97]:
#Connect to database
conn = psycopg2.connect(database='cityData', host='localhost',user = 'postgres', password="DS17")
cur = conn.cursor() 

In [ ]:
#Create Table for results

cur.execute("CREATE TABLE seccresults (idsecc BIGINT  PRIMARY KEY,idseccmad BIGINT)")

#Send Commit

conn.commit()

#### Data from Censo 2011

In [76]:
#%%time
#Select all data except Madrid city data
#cur.execute("SELECT * FROM secc,seccdetail  where  seccdetail.idsecc=secc.idsecc and seccdetail.idccaa <> 13 and seccdetail.codmun <> 79 ")
cur.execute("SELECT * FROM secc")

ver = cur.fetchall()
indexed_df_secc = pd.DataFrame(ver).set_index([0])
#indexed_df_secc = indexed_df_secc.iloc[:,:145]
print indexed_df_secc.shape
indexed_df_secc.head()

(35917, 145)


,1,2,3,4,5,6,7,8,9,10,...,136,137,138,139,140,141,142,143,144,145
0,,,,,,,,,,,,,,,,,,,,,
10400101001,1460,760,695,170,930,360,1300,155,0,5,...,90,80,100,595,140,215,120,85,35,0
10400201001,1350,700,655,150,870,335,1270,80,0,0,...,135,50,50,550,125,170,145,90,15,0
10400301001,765,325,445,70,590,105,740,0,0,0,...,0,0,0,295,0,80,0,80,0,0
10400301002,1750,800,950,335,1140,275,1695,0,45,0,...,0,0,0,650,105,230,105,175,30,0
10400301003,1815,780,1035,470,1120,225,1660,0,10,145,...,0,0,0,610,50,170,180,170,20,15


In [77]:
#Normalize Data
X_normalized_indexed_df_secc = preprocessing.normalize(indexed_df_secc, norm='l1')
X_normalizeddf_seccL2 = preprocessing.normalize(indexed_df_secc, norm='l2')
ix1 =indexed_df_secc.index
col1 = indexed_df_secc.columns
df__normalized_indexed_secc=pd.DataFrame(X_normalized_indexed_df_secc, index=[ix1] ,columns=[col1])
df__normalized_indexed_secc.head()

,1,2,3,4,5,6,7,8,9,10,...,136,137,138,139,140,141,142,143,144,145
0,,,,,,,,,,,,,,,,,,,,,
10400101001,0.053795,0.028003,0.025608,0.006264,0.034267,0.013265,0.047900,0.005711,0.000000,0.000184,...,0.003316,0.002948,0.003685,0.021923,0.005158,0.007922,0.004422,0.003132,0.001290,0.000000
10400201001,0.055158,0.028601,0.026762,0.006129,0.035546,0.013687,0.051890,0.003269,0.000000,0.000000,...,0.005516,0.002043,0.002043,0.022472,0.005107,0.006946,0.005924,0.003677,0.000613,0.000000
10400301001,0.057977,0.024631,0.033725,0.005305,0.044714,0.007958,0.056082,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.022357,0.000000,0.006063,0.000000,0.006063,0.000000,0.000000
10400301002,0.057227,0.026161,0.031066,0.010955,0.037279,0.008993,0.055428,0.000000,0.001472,0.000000,...,0.000000,0.000000,0.000000,0.021256,0.003434,0.007521,0.003434,0.005723,0.000981,0.000000
10400301003,0.056192,0.024149,0.032043,0.014551,0.034675,0.006966,0.051393,0.000000,0.000310,0.004489,...,0.000000,0.000000,0.000000,0.018885,0.001548,0.005263,0.005573,0.005263,0.000619,0.000464


In [78]:
#%time
#Select data from Madrid
cur.execute("SELECT * FROM secc,seccdetail  where  seccdetail.idsecc=secc.idsecc and seccdetail.idccaa = 13 and seccdetail.codmun = 79 ")
ver =  cur.fetchall()
indexed_df_secc_mad = pd.DataFrame(ver).set_index([0])
indexed_df_secc_mad = indexed_df_secc_mad.iloc[:,:145]
indexed_df_secc_mad.head()



,1,2,3,4,5,6,7,8,9,10,...,136,137,138,139,140,141,142,143,144,145
0,,,,,,,,,,,,,,,,,,,,,
132807901001,1205,480,725,150,700,355,955,50,0,0,...,0,0,75,490,170,160,65,50,0,50
132807901002,855,355,505,75,665,115,720,0,0,0,...,0,0,0,485,195,240,0,0,0,0
132807901003,1795,880,920,130,1350,320,1295,110,0,10,...,25,0,65,920,380,320,115,110,0,0
132807901004,1070,495,575,130,785,150,840,0,0,0,...,0,0,0,550,300,70,130,30,0,20
132807901006,1950,895,1055,200,1305,440,1800,35,0,0,...,10,70,25,960,455,210,180,70,20,30


In [79]:
#Normalize Data
X_normalized_indexed_df_mad = preprocessing.normalize(indexed_df_secc_mad, norm='l1')
X_normalizeddf_seccL2 = preprocessing.normalize(indexed_df_secc_mad, norm='l2')
ix =indexed_df_secc_mad.index
col = indexed_df_secc_mad.columns
df__normalized_indexed_mad=pd.DataFrame(X_normalized_indexed_df_mad, index=[ix] ,columns=[col])
df__normalized_indexed_mad.tail()

,1,2,3,4,5,6,7,8,9,10,...,136,137,138,139,140,141,142,143,144,145
0,,,,,,,,,,,,,,,,,,,,,
132807921026,0.056933,0.028306,0.028626,0.010395,0.042540,0.003998,0.044938,0.001279,0.000640,0.00032,...,0.000000,0.000000,0.000000,0.021110,0.002559,0.009435,0.002879,0.004958,0.000480,0.000640
132807921027,0.056758,0.029227,0.027431,0.014963,0.038504,0.003192,0.052070,0.001895,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.020549,0.003691,0.005287,0.004489,0.006384,0.000698,0.000000
132807921028,0.055092,0.026549,0.028543,0.011689,0.040413,0.002990,0.049112,0.003625,0.000181,0.00000,...,0.001631,0.000816,0.000362,0.023831,0.007521,0.007521,0.003896,0.004078,0.000362,0.000453
132807921029,0.058572,0.028635,0.029937,0.021569,0.034771,0.002417,0.055969,0.001673,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.018036,0.001859,0.003533,0.004463,0.005020,0.003347,0.000000
132807921030,0.055866,0.025372,0.030261,0.014898,0.035847,0.005121,0.049348,0.001164,0.000466,0.00000,...,0.000000,0.000000,0.000000,0.021648,0.004888,0.005587,0.005354,0.004888,0.000466,0.000466


## <p style="text-align: center;">Processing data </p>
<p style="text-align: center;">We use cosine similarity to find the distances between vectors.</p>
$$sim (A,B) = cos(\phi)= \frac{A \cdot B}{||A|| \cdot ||B||}$$

In [88]:
#Processing data 

#Cosine Similarity
from sklearn.metrics.pairwise import cosine_similarity
func =cosine_similarity(df__normalized_indexed_mad,df__normalized_indexed_secc)
dfunc = pd.DataFrame(func, index=[ix],columns=[ix1])
dfunc.head()

,10400101001,10400201001,10400301001,10400301002,10400301003,10400301004,10400301005,10400301006,10400301007,10400302001,...,195200108005,195200108006,195200108007,195200108008,195200108009,195200108010,195200108011,195200108012,195200108013,195200108014
0,,,,,,,,,,,,,,,,,,,,,
132807901001,0.934543,0.924619,0.920835,0.929042,0.935487,0.912305,0.932546,0.943995,0.936364,0.907335,...,0.916564,0.925027,0.950778,0.925131,0.924339,0.940986,0.925446,0.918390,0.942421,0.909524
132807901002,0.913229,0.911719,0.919057,0.917211,0.908282,0.911169,0.911315,0.936162,0.924441,0.905099,...,0.921695,0.917598,0.941159,0.918170,0.913000,0.939633,0.919235,0.898468,0.957279,0.909126
132807901003,0.884210,0.871883,0.869689,0.879529,0.877139,0.874581,0.881262,0.903635,0.885500,0.878117,...,0.881960,0.880114,0.924074,0.886789,0.877228,0.903525,0.897447,0.880922,0.932803,0.859487
132807901004,0.929373,0.923661,0.924825,0.933692,0.932181,0.925989,0.931921,0.951327,0.932145,0.924679,...,0.931130,0.930415,0.961439,0.933414,0.925305,0.952669,0.944083,0.933993,0.962687,0.913605
132807901006,0.953319,0.956426,0.952753,0.960700,0.951667,0.947766,0.956929,0.971046,0.958877,0.939831,...,0.959909,0.963854,0.975012,0.961946,0.950352,0.976015,0.954018,0.942500,0.976018,0.952201


In [98]:
#Select the maximum value

df =pd.DataFrame(dfunc.idxmax())
df.index.names = ['Secc']
df.head(30)


,0
Secc,
10400101001,132807910063
10400201001,132807913160
10400301001,132807912020
10400301002,132807917096
10400301003,132807912016
10400301004,132807918026
10400301005,132807917068
10400301006,132807913198
10400301007,132807912086


### Upload Data to DDBB

In [85]:
%%time

#Divided into three parts to avoid memory problems

df= df[:10000] 
#Prepare SQL string
def check_sql_string(sql, a):
    unique = "%PARAMETER%"*len(df)
    sql = sql.replace("?", unique)
    for v in a: sql = str(sql.replace(unique, repr(v), 1))
    return sql
a=df.to_records().tolist()
rw= ', ?'*(len(df)-1)+';'
q = """INSERT INTO seccresults (idsecc,idseccmad) VALUES ?""" +rw 

#test SQL
#print (check_sql_string(q,a))

#Upload Data
cur.execute(check_sql_string(q,a))
conn.commit()

CPU times: user 19min 20s, sys: 25min 17s, total: 44min 38s
Wall time: 44min 51s


In [91]:
%%time

#Divided into three parts to avoid memory problems

df= df[10000:25000] 
#Prepare SQL string
def check_sql_string(sql, a):
    unique = "%PARAMETER%"*len(df)
    sql = sql.replace("?", unique)
    for v in a: sql = str(sql.replace(unique, repr(v), 1))
    return sql
a=df.to_records().tolist()
rw= ', ?'*(len(df)-1)+';'
q = """INSERT INTO seccresults (idsecc,idseccmad) VALUES ?""" +rw 

#test SQL
#print (check_sql_string(q,a))

#Upload Data
cur.execute(check_sql_string(q,a))
conn.commit()

CPU times: user 1h 4min 59s, sys: 1h 26min 11s, total: 2h 31min 10s
Wall time: 2h 31min 59s


In [99]:
%%time

#Divided into three parts to avoid memory problems

df= df[25000:] 
#Prepare SQL string
def check_sql_string(sql, a):
    unique = "%PARAMETER%"*len(df)
    sql = sql.replace("?", unique)
    for v in a: sql = str(sql.replace(unique, repr(v), 1))
    return sql
a=df.to_records().tolist()
rw= ', ?'*(len(df)-1)+';'
q = """INSERT INTO seccresults (idsecc,idseccmad) VALUES ?""" +rw 

#test SQL
#print (check_sql_string(q,a))

#Upload Data
cur.execute(check_sql_string(q,a))
conn.commit()

CPU times: user 25min 1s, sys: 33min 16s, total: 58min 17s
Wall time: 58min 36s
